In [2]:
import re
import sys
import csv
import pandas as pd

PATHS:

In [7]:
#hate speech:
HATE_RAW_PATH = '../datasets/hate/raw/'
HATE_INTERIM_PATH = '../datasets/hate/interim/'

#emoji:


FILES:

In [13]:
HATE_TEST = 'test_text.txt'
HATE_INTERIM_TEST = 'seperated.csv'

FUNCTIONS:

In [9]:
def tokenise(line):
    # Initialise lists
    tokens = []
    unmatchable = []

    # Compile patterns for speedup
    token_pat = re.compile(r'\w+|#|\'|@')
    skippable_pat = re.compile(r'[.,]+')  # typically spaces

    # As long as there's any material left...
    while line:
        # Try finding a skippable token delimiter first.
        skippable_match = re.search(skippable_pat, line)
        if skippable_match and skippable_match.start() == 0:
            # If there is one at the beginning of the line, just skip it.
            line = line[skippable_match.end():]
        else:
            # Else try finding a real token.
            token_match = re.search(token_pat, line)
            #print(token_match)
            if token_match and token_match.start() == 0:
                #print(line[token_match.start():token_match.end()])
                if line[token_match.start():token_match.end()] == '#': #keep hash tags together and seperate
                    try:
                        token_match2 = re.search(token_pat, line[1:])
                        tokens.append(line[:token_match2.end()+1])
                        line = line[token_match2.end()+1:]
                    except:
                        line = line[token_match.end():]

                elif line[token_match.start():token_match.end()] == '@': #keep @ tags together and seperate
                    try:
                        token_match2 = re.search(token_pat, line[1:])
                        tokens.append(line[:token_match2.end()+1])
                        line = line[token_match2.end()+1:]
                    except:
                        line = line[token_match.end():]

                elif line[token_match.start():token_match.end()] == "'": #handle contractions as a single word
                    try:
                        token_match2 = re.search(token_pat, line[1:])
                        if ' ' in line[token_match2.start():token_match2.end()]:
                            line = line[token_match.end():]
                        
                        else: 
                            tokens.append(line[:token_match2.end()+1])
                            line = line[token_match2.end()+1:]
                    except:
                        line = line[token_match.end():]

                # If there is one at the beginning of the line, tokenise it.
                else:
                    tokens.append(line[:token_match.end()])
                    line = line[token_match.end():]
            else:
                # Else there is unmatchable material here.
                # It ends where a skippable or token match starts, or at the end of the line.
                unmatchable_end = len(line)
                if skippable_match:
                    unmatchable_end = skippable_match.start()
                if token_match:
                    unmatchable_end = min(unmatchable_end, token_match.start())
                # Add it to unmatchable and discard from line.
                unmatchable.append(line[:unmatchable_end])
                line = line[unmatchable_end:]

    final_tokens = []

    while len(tokens) > 0:
        temp1 = tokens.pop(0)
        try:
            temp2 = tokens.pop(0)
            if temp2[0] == "'":
                temp1 += temp2
                final_tokens.insert(0, temp1)
                #print('a', temp1)
            else:
                final_tokens.insert(0, temp1)
                tokens.insert(0, temp2)
                #print('b', temp1, temp2)
        except:
            final_tokens.insert(0, temp1)
            #print('d', temp1)
        
    final_tokens = final_tokens[::-1]

    #print(final_tokens)
    #print(unmatchable)
    return final_tokens

In [18]:
f = open(HATE_RAW_PATH + HATE_TEST, "r", encoding="utf-8")
final = []
for line in f:
    final.append(tokenise(line))
f.close()

with open(HATE_INTERIM_PATH+HATE_INTERIM_TEST, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerows(final)

In [24]:
final

[['@user',
  'you',
  'are',
  'correct',
  'that',
  'Reid',
  'certainly',
  'is',
  'a',
  'weasel',
  'Sadly',
  "we've",
  'got',
  'our',
  'own',
  'weasels',
  '@user',
  'Sen',
  'McConnell',
  '@user',
  'The',
  'corrupt',
  'Mueller',
  'investigation',
  'w',
  'be',
  'STOPPED',
  'if',
  'those',
  '3',
  'did',
  'their',
  'jobs',
  '#MAGA',
  '#KAG',
  '#POTUS',
  '#Trump',
  '#NEWS',
  '#VoteRed',
  '#NoDACA',
  '#USA'],
 ['Whoever', 'just', 'unfollowed', 'me', 'you', 'a', 'bitch'],
 ['@user',
  '@user',
  'Those',
  'People',
  'Invaded',
  'Us',
  'They',
  'DO',
  'NOT',
  'BELING',
  'HERE',
  'HAVE',
  'NO',
  'RIGHTS',
  'Its',
  '#AmericaFIRST',
  'Open',
  'Your',
  'House',
  'To',
  'Them',
  'If',
  'Your',
  'That',
  'IGNORANT',
  'Yes',
  'Im',
  'A',
  '#Christian',
  'Too',
  '#NODACA'],
 ['stop',
  'JUDGING',
  'bitches',
  'by',
  'there',
  'cover',
  'jus',
  'cuz',
  'she',
  'bad',
  "don't",
  'mean',
  "she's",
  'a',
  'catch',
  'shawdy',
  